In [9]:
import re
from datetime import datetime
from datetime import timedelta
import base64
import requests
import json
import hashlib 
import time

In [ ]:
db = dataset.connect('sqlite:///../data/db/data.sqlite3')

#TODO 1: edit data/db/data.sqlite3 and add the sites you want to scrape into sites table
site_table = db["sites"]
run_name = "run2_" #leave this as it is

for site_row in site_table:
	state = site_row['state']
	city = site_row['city']
	site = site_row['site']
	site_name = site_row['site_name']
	label = state.lower().replace(" ","") +"_"+city.lower().replace(" ","-")+"_"+site+"_"
	table = db["request_status_data"]

	fromDate = "23-07-2020"  #TODO 2: starting date
	endDate = "20-07-2020"   #TODO 3: ending date, will be next day after that day
	how_many_days = 2

	toDate = ""
	objFromDate = datetime.strptime(fromDate, "%d-%m-%Y" )
	time_part = " T00:00:00Z"
	time_part_end = " T23:59:00Z"
	status_code = 1

	while (objFromDate < datetime.strptime(endDate, "%d-%m-%Y" )):
		print("####################################################")

		objToDate = objFromDate +timedelta(days=how_many_days)

		fromDate=objFromDate.strftime("%d-%m-%Y")+time_part
		toDate=objToDate.strftime("%d-%m-%Y")+time_part

		print(fromDate)
		print(toDate)


		query_name = run_name + label + objFromDate.strftime("%Y%m%d")

		print(query_name)

		#prompt_pm25='{"draw":1,"columns":[{"data":0,"name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}}],"order":[],"start":0,"length":50,"search":{"value":"","regex":false},"filtersToApply":{"parameter_list":[{"id":7,"itemName":"PM2.5","itemValue":"parameter_193"}],"criteria":"4 Hours","reportFormat":"Tabular","fromDate":"'+fromDate+'","toDate":"'+toDate+'","state":"'+state+'","city":"'+city+'","station":"'+site+'","parameter":["parameter_193"],"parameterNames":["PM2.5"]},"pagination":1}'
		prompt_both='{"draw":2,"columns":[{"data":0,"name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}}],"order":[],"start":10,"length":10,"search":{"value":"","regex":false},"filtersToApply":{"parameter_list":[{"id":0,"itemName":"PM2.5","itemValue":"parameter_193"},{"id":1,"itemName":"PM10","itemValue":"parameter_215"}],"criteria":"4 Hours","reportFormat":"Tabular","fromDate":"'+fromDate+'","toDate":"'+toDate+'","state":"'+state+'","city":"'+city+'","station":"'+site+'","parameter":["parameter_193","parameter_215"],"parameterNames":["PM2.5","PM10"]},"pagination":1}'


		data_to_encode = prompt_both
		encoded_data = base64.b64encode(data_to_encode)
		print data_to_encode

		row_exists = table.find_one(query_name=query_name)
		if row_exists:
			print "EXISTS SO GO TO NEXT"
		else:
			row = {}
			row['query_name'] = query_name 
			row['fromDate'] = fromDate
			row['toDate'] = toDate
			row['state'] = state
			row['city'] = city
			row['site'] = site
			row['site_name'] = site_name
			row['data_to_encode'] = data_to_encode
			row['encoded_data'] = encoded_data
			row['status_code'] = status_code		
			row['parsed'] = 0
			table.insert(row)

		#forward in date for next
		objFromDate = objToDate	
		print "_______________________________________________________________"
		#end while


In [ ]:
import re
from datetime import datetime
from datetime import timedelta
import base64
import requests
import dataset
import json
import hashlib 
import time

db = dataset.connect('sqlite:///../data/db/data.sqlite3')
site_table = db["sites"]
table = db["request_status_data"]
status_code = 404 # 0 for low priority
row_exists = table.find_one(status_code=status_code)

while row_exists: 	
	print "#####################################################"+ str(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
	db.begin()
	query_name =row_exists['query_name'] 
	print "query_name = "+str(query_name)
	encoded_data = row_exists['encoded_data'] 

	headers = {'Origin': 'https://app.cpcbccr.com'}
	headers['Accept-Encoding'] ="gzip, deflate, br"
	headers['Accept-Language'] ="en-GB,en-US;q=0.9,en;q=0.8"
	headers['User-Agent'] ="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36"
	headers['Content-Type'] ="application/x-www-form-urlencoded"
	headers['Accept'] ="application/json, text/plain, */*"
	headers['Referer'] ="https://app.cpcbccr.com/ccr/"
	headers['Connection'] ="keep-alive"
	headers['Host'] ="app.cpcbccr.com"

	r = requests.post("https://app.cpcbccr.com/caaqms/fetch_table_data", headers=headers, data=encoded_data, verify=False)
	if r.status_code == 200:
		print "Awesome response code 200"
		json_data = json.dumps(r.json())
		json_data_hash = hashlib.md5(json_data)
		row_exists['json_data'] = json_data
		row_exists['json_data_hash'] = json_data_hash.hexdigest()
		row_exists['status_code'] = r.status_code
	else:
		row_exists['json_data'] = ""
		row_exists['status_code'] = r.status_code

	print "UPDATING"
	table.update(row_exists,['id'])
	db.commit()

	time.sleep(5)
	row_exists = table.find_one(status_code=status_code)
	if row_exists:
		pass
	else:
		break

	#this is for testing	
	#break

print "_______________________________________________________________"

#end while


In [ ]:
import re
from datetime import datetime
from datetime import timedelta
import base64
import requests
import dataset
import json
import hashlib 
import time
import traceback

db = dataset.connect('sqlite:///../data/db/data.sqlite3')
request_status_datatable = db["request_status_data"]
data_table = db["data"]


parse_row  = request_status_datatable.find_one(parsed=0, status_code=200)



while parse_row:
	print "inside while loop"
	db.begin()
	try:
		print str(parse_row)
		json_data = json.loads(parse_row['json_data'])

		data = json_data["data"]
		tabularData = data["tabularData"]
		bodyContent = tabularData["bodyContent"]

		for row in bodyContent:
			insert_row = {}
			insert_row["state"] = parse_row["state"]
			insert_row["city"] = parse_row["city"]
			insert_row["site"] = parse_row["site"]
			insert_row["site_name"] = parse_row["site_name"]
			insert_row["query_name"] = parse_row["query_name"]

			#dateformat : 14-Oct-2017 - 08:00"
			#print str(row)
			if row.has_key("to date"):
				to_date = row["to date"]
				to_date_array = to_date.split(" - ")
				insert_row["to_date"] = to_date_array[0]
				insert_row["to_time"] = to_date_array[1]

			if row.has_key("from date"):
				from_date = row["from date"]
				from_date_array = from_date.split(" - ")
				insert_row["from_date"] = from_date_array[0]
				insert_row["from_time"] = from_date_array[1]

			if row.has_key("PM2.5"):
				pm25 = row["PM2.5"]
				if pm25 and pm25 !="":
					insert_row["pm25"] = pm25

			if row.has_key("PM10"):
				pm10 = row["PM10"]
				if pm10 and pm10 !="":
					insert_row["pm10"] = pm10


			print str(insert_row)
			data_table.insert(insert_row)
			#parsed
			parse_row["parsed"]=1	

	except Exception, err:
		traceback.print_exc()
		#error in parsing
		parse_row["parsed"]=2	

	#update row to parsed
	request_status_datatable.update(parse_row, ['id'])
	db.commit()

	parse_row  = request_status_datatable.find_one(parsed=0, status_code=200)
	if parse_row:
		pass
	else:
		break